In [1]:
import sys
import numpy as np
import scipy as sp
import pandas as pd
from sklearn import svm
from sklearn import linear_model
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import fbeta_score
sys.path.insert(0, '../../notebooks/libs/')
import FeatureExtraction as FE
from sklearn.feature_extraction.text import TfidfVectorizer
from multiprocessing import Pool

In [2]:
import sys
sys.path.insert(0, '../libs/')
%matplotlib inline

import FeatureExtraction
from lxml import etree

training_xml = '../../dataset/training/pan12-sexual-predator-identification-training-corpus-2012-05-01.xml'
test_xml = '../../dataset/test/pan12-sexual-predator-identification-test-corpus-2012-05-17.xml'

sexual_predator_ids_file = '../../dataset/training/pan12-sexual-predator-identification-training-corpus-predators-2012-05-01.txt'

chat_based_features_csv_train='../../csv/chat_based_features_training.csv'
chat_based_features_csv_test='../../csv/chat_based_features_test.csv'

In [ ]:
#matrix_training, matrix_testing = FE.make_tf_idf_matrix_from_XML(training_xml, test_xml)

In [3]:
documents_training=FE.prepare_for_tf_idf(training_xml,False)
document_testing=FE.prepare_for_tf_idf(test_xml,False)

In [4]:
print(len(documents_training))
print(len(document_testing))

97689
218702


In [214]:
tfidf=TfidfVectorizer(stop_words='english',min_df=3,max_features=3500)
matrix_training=tfidf.fit_transform(documents_training)
matrix_testing=tfidf.transform(document_testing)


In [26]:
train_frame=pd.DataFrame(pd.read_csv(chat_based_features_csv_train))
test_frame=pd.DataFrame(pd.read_csv(chat_based_features_csv_test))

In [ ]:
features=['number of conversation', 'percent of conversations started by the author', 'number of messages sent', 'average percent of lines in conversation', 'number of characters sent by the author']
X_train_chat_based=train_frame.ix[:,features]
y_train=np.ravel(train_frame[[-1]])
X_test_chat_based=test_frame.ix[:,features]
y_test=np.ravel(test_frame[[-1]])

In [63]:
X_train = sp.sparse.hstack((X_train_chat_based, matrix_training))
X_test = sp.sparse.hstack((X_test_chat_based, matrix_testing))
print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)

((97689, 2005), (97689L,), (218702, 2005), (218702L,))


In [50]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler(with_mean=False)
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)


In [140]:

svmc=svm.SVC(kernel='linear',max_iter=100)
lrc=linear_model.LogisticRegression(n_jobs=8,max_iter=100000,penalty='l2')




SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=100, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


In [210]:
#scaler=StandardScaler(with_mean=False)
#matrix_training=scaler.fit_transform(matrix_training)
#matrix_testing=scaler.transform(matrix_testing)

In [215]:
#linearsvm=svm.LinearSVC(max_iter=1000)
#ovaj samo na tf idf daje f-score 59 uz tf idf napravljen sa TfidfVectorizer(stop_words='english',min_df=3,max_features=3500)
classifier=svm.SVC(C=0.4,kernel='linear',max_iter=100)
classifier.fit(matrix_training,y_train)
predicted_test=classifier.predict(matrix_testing)

In [216]:
print(predicted_test[predicted_test==1])
print(predicted_test.shape)
print(y_test.shape)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1]
(218702L,)
(218702L,)


In [217]:
#print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)
print("\ntest statistics")
print('acc',accuracy_score(y_test,predicted_test))
print('rec',recall_score(y_test,predicted_test))
print('prec',precision_score(y_test,predicted_test))
print('f1',f1_score(y_test,predicted_test))
print('f0.5',fbeta_score(y_test,predicted_test,0.5))


test statistics
('acc', 0.99923183144187067)
('rec', 0.48031496062992124)
('prec', 0.77215189873417722)
('f1', 0.59223300970873793)
('f0.5', 0.68848758465011295)
